In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential,layers,datasets,optimizers
import numpy as np
import pandas as pd

In [2]:
#The most frequent words
total_words = 10000
(x_train,y_train),(x_test,y_test) = datasets.imdb.load_data(num_words = total_words)

In [3]:
# max sentence length(padding )
max_reveiw_len = 80 
x_train = keras.preprocessing.sequence.pad_sequences(x_train,maxlen=max_reveiw_len)
x_test = keras.preprocessing.sequence.pad_sequences(x_test,maxlen=max_reveiw_len)

In [4]:
len(x_train[1])

80

In [5]:
batchsz = 128
db_train = tf.data.Dataset.from_tensor_slices((x_train,y_train))
db_train = db_train.shuffle(1000).batch(batch_size=batchsz,drop_remainder=True) #drop the last batch
db_test = tf.data.Dataset.from_tensor_slices((x_test,y_test)) 
db_test = db_test.batch(batch_size=batchsz,drop_remainder=True)

In [6]:
# one batch shape
next(iter(db_train))[0].shape

TensorShape([128, 80])

In [55]:
#rnn cell
class MyRNN(keras.Model):
    def __init__(self,units):
        super(MyRNN,self).__init__()
        self.state0 = [tf.zeros([batchsz,units])]
        self.state1 = [tf.zeros([batchsz,units])]
        # transform text to embedding representation
        # [b,80] -> [b,80,100]
        self.embedding = layers.Embedding(total_words,100,input_length = max_reveiw_len)
        # [b,80,100]  h_dim: 64
        self.rnn_cell0 = layers.SimpleRNNCell(units,dropout = 0.2)
        self.rnn_cell1 = layers.SimpleRNNCell(units,dropout = 0.2)
        # [b,80,100] -> [b,64] -> [b,1] 
        self.outlayer = layers.Dense(1)
    def call(self,inputs,training = None):
        x = inputs
        #[b,80] -> [b,80,100]
        x = self.embedding(x)
        # rnn compute
        # [b,80,100] -> [b,64]
        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x,axis=1): # word: [b,100]
            # h1 = x*wxh +h*whh
            out,state0 = self.rnn_cell0(word,state0,training)
            out,state1 = self.rnn_cell1(out,state1)
          #out: [b,64]
        x = self.outlayer(out)
        #p(y id pos|x)
        prob = tf.sigmoid(x)
        return prob

In [7]:
# LSTM
class MyRNN(keras.Model):
    def __init__(self,units):
        super(MyRNN,self).__init__()
        self.state0 = [tf.zeros([batchsz,units]),tf.zeros([batchsz,units])]
        self.state1 = [tf.zeros([batchsz,units]),tf.zeros([batchsz,units])]
        # transform text to embedding representation
        # [b,80] -> [b,80,100]
        self.embedding = layers.Embedding(total_words,100,input_length = max_reveiw_len)
        # [b,80,100]  h_dim: 64
        self.rnn_cell0 = layers.LSTMCell(units,dropout = 0.5)
        self.rnn_cell1 = layers.LSTMCell(units,dropout = 0.5)
        # [b,80,100] -> [b,64] -> [b,1] 
        self.outlayer = layers.Dense(1)
    def call(self,inputs,training = None):
        x = inputs
        #[b,80] -> [b,80,100]
        x = self.embedding(x)
        # rnn compute
        # [b,80,100] -> [b,64]
        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x,axis=1): # word: [b,100]
            # h1 = x*wxh +h*whh
            out,state0 = self.rnn_cell0(word,state0,training)
            out,state1 = self.rnn_cell1(out,state1)
          #out: [b,64]
        x = self.outlayer(out)
        #p(y id pos|x)
        prob = tf.sigmoid(x)
        return prob

In [25]:
units = 64
epochs = 4
model = MyRNN(units)
model.compile(optimizer=optimizers.Adam(0.001),
             loss = tf.losses.BinaryCrossentropy(),
             metrics=['accuracy'])
model.fit(db_train,epochs = epochs, validation_data=db_test)
model.evaluate(db_test)

Epoch 1/4
195/195 [==============================] - 37s 123ms/step - loss: 0.5127 - accuracy: 0.7288 - val_loss: 0.3748 - val_accuracy: 0.8335
Epoch 2/4
195/195 [==============================] - 21s 107ms/step - loss: 0.3239 - accuracy: 0.8637 - val_loss: 0.3710 - val_accuracy: 0.8360
Epoch 3/4
195/195 [==============================] - 21s 107ms/step - loss: 0.2752 - accuracy: 0.8895 - val_loss: 0.3725 - val_accuracy: 0.8369
Epoch 4/4
195/195 [==============================] - 6s 32ms/step - loss: 0.4731 - accuracy: 0.8236


[0.47312843799591064, 0.823637843132019]

In [24]:
# layer method
class MyRNN(keras.Model):
    def __init__(self,units):
        super(MyRNN,self).__init__()
      
        # transform text to embedding representation
        # [b,80] -> [b,80,100]
        self.embedding = layers.Embedding(total_words,100,input_length = max_reveiw_len)
        # [b,80,100]  h_dim: 64
        self.rnn = keras.Sequential([
#             layers.SimpleRNN(units,dropout = 0.5,return_sequences = True, unroll = True),
#             layers.SimpleRNN(units,dropout = 0.5,unroll = True)
            
#             layers.LSTM(units,dropout = 0.5,return_sequences = True, unroll = True)
#             layers.LSTM(units,dropout = 0.5,unroll = True)
            layers.GRU(units,dropout = 0.5,return_sequences = True, unroll = True),
            layers.GRU(units,dropout = 0.5,unroll = True)
        ])
        
        # [b,80,100] -> [b,64] -> [b,1] 
        self.outlayer = layers.Dense(1)
    def call(self,inputs,training = None):
        x = inputs
        #[b,80] -> [b,80,100]
        x = self.embedding(x)
        # rnn compute
        # [b,80,100] -> [b,64]
        x = self.rnn(x)
          #out: [b,64]
        x = self.outlayer(x)
        #p(y id pos|x)
        prob = tf.sigmoid(x)
        return prob